# Convert a Keras model to TensorFlow.js format

In [2]:
sequence_length = 30  # Number of frames per sequence
shape_no_face = (sequence_length, 258)
num_classes = 4
actions = ['hallo','du','idle','a']  # List all your actions including idle
DATA_PATH = 'CollectionBase/MP_Data/30frame_no_face_+idle'  # Update with your data path
landmarks = [False,True,True,True] # Select True/False for different landmarks (face, pose, left hand, right hand)

In [4]:
import h5py
import tensorflow as tf

# Load your model
model = tf.keras.models.load_model('ai.h5')

# Print the summary to inspect the layers and weights
model.summary()

# Optionally, rename duplicate weights
for layer in model.layers:
    if hasattr(layer, 'weights'):
        for i, weight in enumerate(layer.weights):
            if 'gamma' in weight.name:
                new_name = f"{weight.name.split(':')[0]}_{i}"
                weight._handle_name = new_name

# Save the model again after renaming weights
model.save('ai_fixed.h5')


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ masking_1 (Masking)             │ (None, 30, 258)           │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ lstm_6 (LSTM)                   │ (None, 30, 128)           │    198,144 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_8 (Dropout)             │ (None, 30, 128)           │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization_6           │ (None, 30, 128)           │        512 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ lstm_7 (LSTM)                   │ (None, 30, 128)           │    131,584 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_9 (Dropout)             │ (None, 30, 128)           │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization_7           │ (None, 30, 128)           │        512 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ lstm_8 (LSTM)                   │ (None, 128)               │    131,584 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_10 (Dropout)            │ (None, 128)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization_8           │ (None, 128)               │        512 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_4 (Dense)                 │ (None, 256)               │     33,024 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_11 (Dropout)            │ (None, 256)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_5 (Dense)                 │ (None, 4)                 │      1,028 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 496,902 (1.90 MB)

 Trainable params: 496,132 (1.89 MB)

 Non-trainable params: 768 (3.00 KB)

 Optimizer params: 2 (12.00 B)

In [6]:

import numpy as np
from keras.src.layers import Masking
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
import os
import tensorflowjs as tfjs

l2_factor = 0.01

model = Sequential()

# Input layer
model.add(Masking(mask_value=0., input_shape=shape_no_face))  # Adjust input shape accordingly

# Adding LSTM layers with L2 regularization
model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(LSTM(128, kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

# Fully connected layer with L2 regularization
model.add(Dense(256, activation='relu', kernel_regularizer=l2(l2_factor)))
model.add(Dropout(0.3))

# Output layer for num_classes with L2 regularization
model.add(Dense(num_classes, activation='softmax', kernel_regularizer=l2(l2_factor)))


# Load the weights into the model
model.load_weights('ai.h5')

# Save the model in TensorFlow.js format
tfjs.converters.save_keras_model(model, './output_directory')

failed to lookup keras version from the file,
    this is likely a weight only file


Exception: Error dumping weights, duplicate weight name gamma